In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

random_state = 67

bank_marketing = fetch_ucirepo(id=222)
X = bank_marketing.data.features
y = bank_marketing.data.targets

# EDA

### Target

In [ ]:
print(type(y))
print(y.shape)
print(y.dtypes)

In [ ]:
y_series = y.iloc[:,0]
print(y_series.value_counts())
print(y_series.value_counts(normalize=True))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(data = y, x = y_series, palette='hls')
plt.title("Distribution of y")
plt.show()
plt.close()

Target jest mocno niezbalansowany, przez co podczas liczenia metryk nie będziemy opierać się na accuracy(baseline ~0.88 dla 'no'), będziemy raczej chcieli patrzeć na recall/f1 dla 'yes'. Threshold 0.5 też nie będzie dobry przy takim rozkładzie y.

## Features

In [ ]:
X_eda = X.copy()
print(X_eda.head())
print(X_eda.dtypes)
print(X_eda.shape)

### Missing values

In [ ]:
print(X_eda.isna().sum())

In [ ]:
missing_cols = X_eda.isna().sum().to_frame()
missing_cols = missing_cols[missing_cols.loc[:,0] > 0].index
for col in missing_cols:
    print('unknown' in X_eda.loc[:,col].values)

Kolumny gdzie są brakujące dane maja dtype = object oraz nie mają w sobie defaultowo 'unknown', także zamienimy brakujące wartośći na 'unknown'

In [ ]:
X_eda.loc[:,missing_cols] = X_eda.loc[:,missing_cols].fillna('unknown')
print(X_eda.isna().sum())

### Outliers

In [ ]:
num_cols = X_eda.select_dtypes('number').columns.tolist()
cat_cols = X_eda.select_dtypes('object').columns.tolist()
print(X_eda.loc[:,num_cols].describe())

Duration mówi nam o czasie rozmowy, a znamy ją dopiero po zakończeniu jej, dlatego usuwamy ją.

In [ ]:
df_with_y = X_eda.copy()
df_with_y['y'] = y_series
if 'duration' in df_with_y.columns:
    df_with_y.drop(columns=['duration'], inplace=True)

num_cols_without_duration = df_with_y.select_dtypes('number').columns.tolist()

for col in num_cols_without_duration:
    sns.violinplot(data = df_with_y, x="y", y=col, cut = 0, inner='quartile')
    plt.title(f"Distribution of {col}")
    plt.show()
    plt.close()

In [ ]:
df_pdays = df_with_y[df_with_y['pdays'] != -1].copy()
sns.violinplot(data=df_pdays, x='y', y='pdays', cut=0, inner='quartile')
plt.title('Pdays without -1')
plt.show()
plt.close()

### Corelations

In [ ]:
from sklearn.feature_selection import mutual_info_classif

y_bin = (y_series == "yes").astype('int')

mi_num = mutual_info_classif(X_eda.loc[:,num_cols_without_duration], y=y_bin, random_state=random_state)
mi_num = pd.Series(mi_num, index=num_cols_without_duration).sort_values(ascending=False)
print(mi_num.head())

In [ ]:
from scipy.stats import chi2_contingency

def cramer_v(x,y):
    ct = pd.crosstab(x,y)
    chi2, p, dof, freq = chi2_contingency(ct)
    n = ct.to_numpy().sum()
    r, k = ct.shape
    v = np.sqrt((chi2/n)/(min(r-1,k-1)))
    return v, chi2, p, dof, r, k

rows = []
for col in cat_cols:
    v, chi2, p, dof, r, k = cramer_v(X_eda.loc[:,col], y_bin)
    rows.append({
        "feature": col,
        "cramers_v": v,
        "chi2": chi2,
        "p-value": p,
        "n_categories": r
    })

cramers_rank = pd.DataFrame(rows).sort_values(by="cramers_v", ascending=False)
print(cramers_rank.head(10))

## EDA summary

- Niezbalansowany target (~12% yes)
- Duration znane dopiero po rozmowie
- Braki danych w kolumnach kategorycznych
- Specjalna flaga -1 dla pdays jeżeli ktoś nie był kontaktowany
- Kolumny numeryczne mają duże zakresy, dla modeli liniowych można użyć scalerów
- Z kategorycznych największy związek z targetem ma poutcome, month, contact, housing, job
- Katerogyczne wymagają OneHotEncdoing

# Preprocessing

In [ ]:
X_prep = X.copy()

if 'duration' in X_prep.columns:
    X_prep.drop(columns=['duration'], inplace=True)

if 'pdays' in X_prep.columns:
    X_prep['prev_contacted'] = (X_prep['pdays'] != -1).astype('int')
    X_prep['pdays_clean'] = X_prep['pdays'].where(X_prep['pdays'] != -1, np.nan)
    X_prep.drop(columns=['pdays'], inplace=True)

print(X_prep.dtypes)
print(X_prep.shape)
print(X_prep.head())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_prep,
    y_bin,
    test_size=0.2,
    stratify=y_bin,
    random_state=random_state
)

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y_train,
    stratify=y_train,
    random_state= random_state,
    test_size=0.2
)

num_cols = X_tr.select_dtypes('number').columns.tolist()
cat_cols = X_tr.select_dtypes('object').columns.tolist()

num_skew_pos_cols = ['campaign', 'previous']
num_skew_signed_cols = ['balance']
num_rest_cols = [c for c in num_cols if c not in num_skew_pos_cols + num_skew_signed_cols]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

def log1p(X):
    X = X.copy()
    X[:,:] = np.log1p(X)
    return X

def signed_log1p(X):
    X = X.copy()
    X[:, :] = np.sign(X) * np.log1p(np.abs(X))
    return X

num_skew_pos_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(log1p)),
    ("scaler", RobustScaler())
])

num_skew_signed_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("slog", FunctionTransformer(signed_log1p)),
    ("scaler", RobustScaler())
])

num_rest_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', min_frequency=20))
])

preprocess = ColumnTransformer(transformers=[
    ('num_rest', num_rest_pipe, num_rest_cols),
    ('cat', cat_pipe, cat_cols),
    ("skew_pos", num_skew_pos_pipe, num_skew_pos_cols),
    ("skew_signed", num_skew_signed_pipe, num_skew_signed_cols)
])

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

pipe_lr = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', LogisticRegression(max_iter = 5000))
])

C_grid = np.logspace(-4,1,12)

params_lr = [
    {"model__solver": ["lbfgs"], "model__C": C_grid, "model__l1_ratio": [0], "model__class_weight": [None, "balanced"]},
    {"model__solver": ["liblinear"], "model__C": C_grid, "model__l1_ratio": [0,1], "model__class_weight": [None, "balanced"]}
]

cv = StratifiedKFold(n_splits =5, shuffle=True, random_state=random_state)

gs_lr = GridSearchCV(
    pipe_lr,
    param_grid=params_lr,
    scoring='average_precision',
    cv= cv,
    n_jobs = -1
)

gs_lr.fit(X_tr, y_tr)

best_lr = gs_lr.best_estimator_

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, average_precision_score

proba_lr = best_lr.predict_proba(X_val)[:,1]
thr = np.linspace(0.05, 0.95, 200)

best_thr_lr, best_f1_lr = None, -1
for t in thr:
    pred_lr = (proba_lr >= t).astype(int)
    f1 = f1_score(y_val, pred_lr)
    if f1 > best_f1_lr:
        best_thr_lr, best_f1_lr = t, f1

pred_val_lr = (proba_lr >= best_thr_lr).astype(int)

In [ ]:
print(f"Threshold = {best_thr_lr}")
print(f"Precision = {precision_score(y_val, pred_val_lr)}")
print(f"Recall = {recall_score(y_val, pred_val_lr)}")
print(f"F1 = {best_f1_lr}")

In [ ]:
final_lr = gs_lr.best_estimator_
final_lr.fit(X_train, y_train)
final_proba_lr = final_lr.predict_proba(X_test)[:,1]
final_pred_lr = (final_proba_lr >= best_thr_lr).astype(int)

print("---- Final model -----")
print(f"AP = {average_precision_score(y_test, final_proba_lr)}")
print(f"F1 = {f1_score(y_test, final_pred_lr)}")
print(f"Recall = {recall_score(y_test, final_pred_lr)}")
print(f"Precision = {precision_score(y_test, final_pred_lr)}")

In [ ]:
print("----- Classification raport -----")
print(classification_report(y_test, final_pred_lr, target_names=["no", "yes"], zero_division=0))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_prep = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', min_frequency=20), cat_cols),
    ('num', num_rest_pipe, num_cols)
])

pipe_rf = Pipeline(steps=[
    ('preprocess', rf_prep),
    ('rf', RandomForestClassifier(random_state=random_state, n_jobs=-1, class_weight="balanced_subsample"))
])

rf_grid_params = {
    "rf__n_estimators": [400,425,450],
    "rf__min_samples_leaf": [5,10,15], 
    "rf__max_depth": [36,40],
    "rf__max_features": ["sqrt"]
}

gs_rf = GridSearchCV(
    pipe_rf,
    param_grid= rf_grid_params,
    scoring='average_precision',
    cv=cv,
    n_jobs = -1,
)

gs_rf.fit(X_tr, y_tr)
best_rf = gs_rf.best_estimator_

In [ ]:
proba_rf = best_rf.predict_proba(X_val)[:,1]

best_thr_rf, best_f1_rf = None, -1
for t in thr:
    pred = (proba_rf >= t).astype(int)
    f1 = f1_score(y_val, pred)
    if f1 > best_f1_rf:
        best_thr_rf, best_f1_rf = t, f1

pred_rf = (proba_rf >= best_thr_rf).astype(int)

In [ ]:
print(f"Best THR = {best_thr_rf}")
print(f"Recall = {recall_score(y_val, pred_rf)}")
print(f"F1 = {best_f1_rf}")
print(f"Prec = {precision_score(y_val, pred_rf)}")

In [ ]:
final_rf = gs_rf.best_estimator_
final_rf.fit(X_train, y_train)
final_proba_rf = final_rf.predict_proba(X_test)[:,1]
final_pred_rf = (final_proba_rf >= best_thr_rf).astype(int)

print("---- Final model -----")
print(f"AP = {average_precision_score(y_test, final_proba_rf)}")
print(f"F1 = {f1_score(y_test, final_pred_rf)}")
print(f"Recall = {recall_score(y_test, final_pred_rf)}")
print(f"Precision = {precision_score(y_test, final_pred_rf)}")
print(classification_report(y_test, final_pred_rf, target_names=["no", "yes"]))

# LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

num_cols_svm_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocess_svm = ColumnTransformer(transformers=[
    ('num', num_cols_svm_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

pipe_svm = Pipeline(steps=[
    ('preprocess', preprocess_svm),
    ('svm', LinearSVC(max_iter = 15000, random_state=random_state))
])

C_grid_svm = np.logspace(-3,1,12)

param_grid_svm = {
    "svm__C": C_grid_svm,
    "svm__class_weight": [None, "balanced"],
    "svm__loss": ["squared_hinge"]
}

gs_svm = GridSearchCV(
    pipe_svm,
    param_grid=param_grid_svm,
    cv=cv,
    n_jobs=-1,
    scoring="average_precision"
)

gs_svm.fit(X_tr, y_tr)

best_svm = gs_svm.best_estimator_

In [ ]:
val_scores = best_svm.decision_function(X_val)

t_min, t_max = np.percentile(val_scores, 1), np.percentile(val_scores, 99)

thr_svm = np.linspace(t_min, t_max, 200)

best_thr_svm, best_f1_svm = None, -1
for t in thr_svm:
    pred = (val_scores >= t).astype(int)
    f1 = f1_score(y_val, pred)
    if f1 > best_f1_svm:
        best_thr_svm, best_f1_svm = t, f1

val_pred = (val_scores >= best_thr_svm).astype(int)

In [ ]:
final_svm = gs_svm.best_estimator_
final_svm.fit(X_train, y_train)
final_proba_svm = final_svm.decision_function(X_test)
final_pred_svm = (final_proba_svm >= best_thr_svm).astype(int)

print("---- Final model -----")
print(f"AP = {average_precision_score(y_test, final_proba_svm)}")
print(f"F1 = {f1_score(y_test, final_pred_svm)}")
print(f"Recall = {recall_score(y_test, final_pred_svm)}")
print(f"Precision = {precision_score(y_test, final_pred_svm)}")
print(classification_report(y_test, final_pred_svm, target_names=["no", "yes"]))

# SHAP

In [ ]:
import shap

pre = final_rf.named_steps['preprocess']
rf = final_rf.named_steps['rf']

feature_names = pre.get_feature_names_out()

X_train_rf = pre.transform(X_train)
X_test_rf = pre.transform(X_test)
X_bg = pre.transform(X_train.sample(200, random_state=random_state))

In [ ]:
y_true = np.array(y_test)
y_pred = final_rf.predict(X_test)

tp_idx = np.where((y_true == 1) & (y_pred == 1))[0][0]
fn_idx = np.where((y_true == 1) & (y_pred == 0))[0][0]

X_two = X_test_rf[[tp_idx, fn_idx]]

explainer = shap.TreeExplainer(rf, data=X_bg)
shap_values = explainer.shap_values(X_two, check_additivity=False)

if isinstance(shap_values, list):

    sv_yes = shap_values[1]
    base_yes = explainer.expected_value[1]
else:
    sv_yes = shap_values[:, :, 1]
    base_yes = explainer.expected_value[1]

def row_to_dense(X_row):
    return X_row.toarray().ravel() if hasattr(X_row, "toarray") else np.array(X_row).ravel()

x0 = row_to_dense(X_two[0])
exp0 = shap.Explanation(values=sv_yes[0], base_values=base_yes, data=x0, feature_names=feature_names)
shap.plots.waterfall(exp0, max_display=10)

x1 = row_to_dense(X_two[1])
exp1 = shap.Explanation(values=sv_yes[1], base_values=base_yes, data=x1, feature_names=feature_names)
shap.plots.waterfall(exp1, max_display=10)